# zipline MeanReversion Backtest

In the chapter 04, we introduced `Zipline` to simulate the computation of alpha factors from trailing cross-sectional market, fundamental, and alternative data.

Now we will exploit the alpha factors to derive and act on buy and sell signals using the custom MeanReversion factor developed in the last chapter.

## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import numpy as np
import pandas as pd
from pytz import UTC

from logbook import (NestedSetup, NullHandler, Logger, StreamHandler, StderrHandler, 
                     INFO, WARNING, DEBUG, ERROR)

from zipline import run_algorithm
from zipline.api import (attach_pipeline, 
                         date_rules, 
                         time_rules,
                         get_datetime,
                         order_target_percent,
                         pipeline_output, 
                         record, 
                         schedule_function, 
                         get_open_orders, 
                         calendars,
                         set_commission, 
                         set_slippage)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from pyfolio.utils import extract_rets_pos_txn_from_zipline

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('whitegrid')

## Logging Setup

In [4]:
# setup stdout logging
format_string = '[{record.time: %H:%M:%S.%f}]: {record.level_name}: {record.message}'
zipline_logging = NestedSetup([NullHandler(level=DEBUG),
                               StreamHandler(sys.stdout, format_string=format_string, level=INFO),
                               StreamHandler(sys.stderr, level=ERROR)])
zipline_logging.push_application()
log = Logger('Algorithm')

## Algo Settings

In [5]:
# Settings
MONTH = 21
YEAR = 12 * MONTH
N_LONGS = 50
N_SHORTS = 50
VOL_SCREEN = 500

In [6]:
start = pd.Timestamp('2013-01-01', tz=UTC)
end = pd.Timestamp('2017-01-01', tz=UTC)
capital_base = 1e7

## Mean Reversion Factor

In [7]:
class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())

## Create Pipeline

The Pipeline created by the `compute_factors()` method returns a table with a long and a short column for the 25 stocks with the largest negative and positive deviations of their last monthly return from its annual average, normalized by the standard deviation. It also limited the universe to the 500 stocks with the highest average trading volume over the last 30 trading days. 

In [8]:
def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs'  : mean_reversion.bottom(N_LONGS),
                             'shorts' : mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))

Before_trading_start() ensures the daily execution of the pipeline and the recording of the results, including the current prices.

In [9]:
def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')
    record(factor_data=context.factor_data.ranking)
    assets = context.factor_data.index
    record(prices=data.current(assets, 'price'))

## Set up Rebalancing

The new rebalance() method submits trade orders to the exec_trades() method for the assets flagged for long and short positions by the pipeline with equal positive and negative weights. It also divests any current holdings that are no longer included in the factor signals:

In [10]:
def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    assets = factor_data.index
    
    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = context.portfolio.positions.keys() - longs.union(shorts)
    log.info('{} | Longs: {:2.0f} | Shorts: {:2.0f} | {:,.2f}'.format(get_datetime().date(),
                                                                     len(longs), 
                                                                     len(shorts),
                                                                     context.portfolio.portfolio_value))

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS if N_LONGS else 0)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS if N_SHORTS else 0)

In [11]:
def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)

## Initialize Backtest

The `rebalance()` method runs according to `date_rules` and `time_rules` set by the `schedule_function()` utility at the beginning of the week, right after market_open as stipulated by the built-in US_EQUITIES calendar (see docs for details on rules). 

You can also specify a trade commission both in relative terms and as a minimum amount. There is also an option to define slippage, which is the cost of an adverse change in price between trade decision and execution

In [12]:
def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)

    set_commission(us_equities=commission.PerShare(cost=0.00075, 
                                                   min_trade_cost=.01))
    set_slippage(us_equities=slippage.VolumeShareSlippage(volume_limit=0.0025, 
                                                          price_impact=0.01))

## Run Algorithm

The algorithm executes upon calling the run_algorithm() function and returns the backtest performance DataFrame.

In [ ]:
backtest = run_algorithm(start=start,
                         end=end,
                         initialize=initialize,
                         before_trading_start=before_trading_start,
                         bundle='quandl',
                         capital_base=capital_base)

[ 05:12:05.055105]: INFO: 2013-01-07 | Longs: 10 | Shorts:  2 | 10,000,000.00
[ 05:12:08.404631]: INFO: 2013-01-14 | Longs: 13 | Shorts:  3 | 10,027,422.36
[ 05:12:09.025618]: INFO: 2013-01-22 | Longs:  8 | Shorts:  5 | 10,097,663.57
[ 05:12:09.521231]: INFO: 2013-01-28 | Longs:  5 | Shorts: 14 | 10,106,150.06
[ 05:12:10.156233]: INFO: 2013-02-04 | Longs: 12 | Shorts: 11 | 10,135,417.26
[ 05:12:10.791233]: INFO: 2013-02-11 | Longs: 11 | Shorts: 13 | 10,182,644.42
[ 05:12:11.431233]: INFO: 2013-02-19 | Longs: 21 | Shorts: 11 | 10,127,755.45
[ 05:12:11.976232]: INFO: 2013-02-25 | Longs: 16 | Shorts:  8 | 10,180,357.23
[ 05:12:12.649750]: INFO: 2013-03-04 | Longs: 10 | Shorts: 10 | 10,200,300.19
[ 05:12:13.283750]: INFO: 2013-03-11 | Longs: 12 | Shorts:  7 | 10,240,453.13
[ 05:12:13.905750]: INFO: 2013-03-18 | Longs:  7 | Shorts:  7 | 10,150,394.01
[ 05:12:14.522750]: INFO: 2013-03-25 | Longs:  8 | Shorts:  7 | 10,149,029.25
[ 05:12:15.025750]: INFO: 2013-04-01 | Longs:  4 | Shorts:  6 | 

[ 05:13:20.384250]: INFO: 2015-01-20 | Longs: 12 | Shorts:  7 | 9,911,930.16
[ 05:13:20.890248]: INFO: 2015-01-26 | Longs: 11 | Shorts:  8 | 9,933,179.35
[ 05:13:21.507248]: INFO: 2015-02-02 | Longs: 18 | Shorts:  7 | 9,874,894.91
[ 05:13:22.140468]: INFO: 2015-02-09 | Longs: 15 | Shorts:  6 | 10,026,955.51
[ 05:13:22.782437]: INFO: 2015-02-17 | Longs: 12 | Shorts: 11 | 10,080,570.15
[ 05:13:23.312468]: INFO: 2015-02-23 | Longs: 13 | Shorts:  7 | 10,087,564.34
[ 05:13:23.937468]: INFO: 2015-03-02 | Longs: 16 | Shorts:  7 | 10,042,612.79
[ 05:13:24.563469]: INFO: 2015-03-09 | Longs: 12 | Shorts:  5 | 9,963,112.34
[ 05:13:25.190437]: INFO: 2015-03-16 | Longs: 15 | Shorts:  2 | 10,023,732.49
[ 05:13:25.813948]: INFO: 2015-03-23 | Longs: 10 | Shorts:  2 | 10,078,627.34
[ 05:13:26.424945]: INFO: 2015-03-30 | Longs: 13 | Shorts:  4 | 10,072,282.97
[ 05:13:26.916975]: INFO: 2015-04-06 | Longs: 12 | Shorts:  3 | 10,123,357.71
[ 05:13:27.512068]: INFO: 2015-04-13 | Longs:  5 | Shorts:  8 | 10,1

## Extract pyfolio Inputs

The `extract_rets_pos_txn_from_zipline` utility provided by `pyfolio` extracts the data used to compute performance metrics.

In [ ]:
returns, positions, transactions = extract_rets_pos_txn_from_zipline(backtest)

## Persist Results for use with `pyfolio`

In [ ]:
with pd.HDFStore('backtests.h5') as store:
    store.put('backtest/equal_weight', backtest)
    store.put('returns/equal_weight', returns)
    store.put('positions/equal_weight', positions)
    store.put('transactions/equal_weight', transactions)

## Plot Results

In [ ]:
fig, axes= plt.subplots(nrows=2, figsize=(14,6))
returns.add(1).cumprod().sub(1).plot(ax=axes[0], title='Cumulative Returns')
transactions.groupby(transactions.dt.dt.day).txn_dollars.sum().cumsum().plot(ax=axes[1], title='Cumulative Transactions')
fig.tight_layout()
sns.despine();

In [ ]:
positions.index = positions.index.date

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(positions.replace(0, np.nan).dropna(how='all', axis=1).T, 
            cmap=sns.diverging_palette(h_neg=20, h_pos=200), ax=ax, center=0);

In [ ]:
positions.head()

In [ ]:
transactions.info()